# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [48]:
# Write your code below.
import os
from glob import glob


In [49]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [51]:
import pandas as pd
import os

import sys
sys.path.append (os.getenv('PRICE_DATA'))

from glob import glob
parquet_files = os.getenv('PRICE_DATA')
df = glob(os.path.join(parquet_files, "*/*.parquet"))

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [55]:
# Write your code below.
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag'] - 1
).assign( 
    hi_lo_range = lambda x: x['High'] - x['Low']
))


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [56]:
# Write your code below.
pandas_df = dd_feat.compute()

# Step 2: Add a rolling average return calculation with a window of 10 days grouped by 'ticker'\n",
pandas_df['rolling_avg_return'] = pandas_df.groupby('ticker', sort =False)['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)

# Display the resulting Pandas DataFrame with the rolling average return\n",
print(pandas_df)


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
IP     2003-01-02  32.695671  33.386753  32.266079  33.181293  15.344514   
IP     2003-01-03  33.013195  33.181293  32.592941  32.891788  15.210627   
IP     2003-01-06  32.835754  34.021801  32.835754  33.704277  15.586359   
IP     2003-01-07  33.648243  34.031139  33.396091  33.881718  15.668420   
IP     2003-01-08  33.573532  33.610886  32.966499  33.087906  15.301325   
...           ...        ...        ...        ...        ...        ...   
BXP    2000-12-22  44.437500  44.500000  44.187500  44.375000  14.991729   
BXP    2000-12-26  44.312500  44.312500  44.000000  44.312500  14.970618   
BXP    2000-12-27  43.687500  44.187500  43.687500  44.187500  15.109095   
BXP    2000-12-28  44.062500  44.875000  43.750000  44.750000  15.301436   
BXP    2000-12-29  44.687500  44.812500  43.187500  43.500000  14.874022   

         Vo

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? No, it could have been calculated in Dask.
+ Would it have been better to do it in Dask? Why? Perhaps it could have been faster and more efficient, since the newer versions of Dask includes the moving average and mean functions.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.